# Lab 3 - Data manipulation

Last lab we spent some time learning how to add, delete, and change columns in a pandas DataFrame. In this lab we will expand some of that functionality to learn how to use indexes, sort data, perform transpositions, and add data. We'll also learn how to join DataFrames together to create a new dataset.

Today we are going to work with a sample of the 2015 Federal Election returns data from Elections Canada. You can find the original data from the [Open Canada data](http://open.canada.ca/data/en/dataset/6a919bd5-491e-466e-a279-00cbf7a8e02c) website. Each row of the data feature the vote totals from each polling station for each party, plus the name and number of the election district, and the name and number of the polling station. 

## Indexing

When you load data into pandas, you often will want to use a unique index for each record. Some datasets include native indexing, such as Twitter. All tweets natively have a unique number assigned by Twitter itself. Indexing is important because it allows us to return to a unique record by referencing a single value.

If you would like to import your dataset which has an index, you can specify which column features the index. In the case of the elections data, I have created a numerical index for each 

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/election-ca-2015-sample.csv', index_col = 0)

In [ ]:
df

The bold numbers on the very left of the DataFrame indicate that this is an index. Indexes will come to be important a bit later when we get into joining. Just remember that an index is a *unique identifier* for each row.

**NB**: The <code>NaN</code> value is a special value in numpy which indicates that there is missing data. Not that the value equals zero, but that no value was provided to the agency. It may mean the same thing here (we can probably assume no value was provided for Bloc Québécois for polling stations in Ontario because there was no candidate there), but that is not always the case.

## Sorting

Sorting is a relatively common activity for looking into data. If we, say, want to see which polling stations had the most votes for the Liberal party, we can do the following.

In [ ]:
df.sort_values('Liberal Party of Canada')

By default this will sort values in ascending order (as in, going up). If we want them to be going down, we can do pass an argument for that.

In [ ]:
df.sort_values('Liberal Party of Canada', ascending = False)

## Type conversion

It is typically the case in which we want to convert a column from one type to another. For instance, in the DataFrame, <code>ElectoralDistrictNumber</code> is an identifier rather than a meaningful quantitative measure. So it may make sense to store it as a string rather than a number. We can do the conversion with <code>astype</code>.

In [ ]:
df['ElectoralDistrictNumber_str'] = df['ElectoralDistrictNumber'].astype(str)

In [ ]:
df.dtypes

## Joining data

![](img/img_innerjoin.gif)

One common task of handling data is joining elements of data from other sources together. Say you have data on Census tracts from one Census table, and you want to connect them with data from another Census table. This is where joining comes in.

pandas uses [SQL-style joins](http://www.sql-join.com/). SQL can get very complicated. For now, what you should know is that the default is an inner join. What that means is that pandas will join two tables together only if the records exist in both table 1 and table 2. pandas also will join, by default, on the DataFrame index. In the example below, the index lists are identical, so both DataFrames will stay the same size.

In [ ]:
df2 = pd.read_csv('data/election-ca-2015-sample-total-rejected.csv', index_col = 0)
df3 = df.join(df2)

In [ ]:
df3

## Grouping: split-apply-combine

![](https://camo.githubusercontent.com/60a1e7e95eaef8f9a99f43335368915eafedda3e/687474703a2f2f7777772e686f66726f652e6e65742f737461743537392f736c696465732f73706c69742d6170706c792d636f6d62696e652e706e67) 

[[image source](https://github.com/ramnathv/rblocks/issues/8)]

One of the more powerful idioms available to you with pandas is the split-apply-combine idiom.

The idea is that you group rows together based on shared values. In the image above, grouping is done on the x variable. Then you apply some function to that value. In this case, it is the mean. Afterward, those values are recombined into a single DataFrame.

The <code>groupby</code> method allows you to do this with ease in Python. You provide a variable you'd like to group on. This produces a <code>groupby</code> object. From there, you can apply built-in functions like mean or even function of your own design.

In [ ]:
gr = df.groupby('Province')

In [ ]:
gr['New Democratic Party'].mean()

In [ ]:
gr['New Democratic Party'].sum()

## Applying functions

Lastly, if we want to apply a function every row of the DataFrame, or perhaps every value of a column, we can use <code>apply</code>. <code>apply</code> tells pandas that it wants to apply a function to every value of the object. For now, let's focus on applying to columns.

Say we want to calculate what percentage of the Liberal vote was achieved at the polling station. We can do that by using column math, like below.

In [ ]:
df3['ratio_liberal'] = df3['Liberal Party of Canada'] / df3['TotalVotes']

However, some of these are going to be <code>NaN</code>, since some stations didn't report any Liberal votes. But those are effectively zero. We can convert those to zero with a function we define, called <code>nanToZero</code>. What this does is check if the value is <code>NaN</code>. If it is, then it returns 0. If not, it returns the original value.

In [ ]:
def nanToZero(x):
    if np.isnan(x):
        return 0
    else:
        return x

Finally, we run apply on the new column to do the conversion.

In [ ]:
df3['ratio_liberal'] = df3['ratio_liberal'].apply(nanToZero)

A more advanced way of doing this would be to create an anonymous inline function using [<code>lambda</code>](https://docs.python.org/3.6/tutorial/controlflow.html#lambda-expressions) plus the [ternary operator](https://docs.python.org/3/faq/programming.html#is-there-an-equivalent-of-c-s-ternary-operator). But this is just a shorthand for not having to write a full function above.

In [ ]:
## alternative way of writing this using lambda + ternary operator
df3['ratio_liberal'] = df3['ratio_liberal'].apply(lambda x: 0 if np.isnan(x) else x)

Lastly, as a preview of next week, we can do a histogram plot of the <code>ratio_liberal</code> variable.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

df3['ratio_liberal'].hist()

**Exercise 1**

You will be working with a sample of tweets which mention Canadian Tire. The file can be found at <code>data/canadian-tire-twitter-sample.csv</code>.

1. Load the file. The index is in column 0.
2. Look at the columns. Sort the data by the datetime it was tweeted.
3. Convert the user IDs to strings.
4. Join the existing DataFrame to a new DataFrame loaded from the file <code>data/canadian-tire-twitter-sample-pos-neg.csv</code>. This new DataFrame includes counts of positive and negative words within the tweet. Make sure to store the resultant DataFrame.
5. You can get create a new column which lists the year-month pair of the tweet like so:

In [ ]:
df['date'] = df['created_at'].str.split(expand = True)[0]
df['month'] = df['date'].str.slice(0, 7)

With this new column, group by month and produce a mean number of positive and negative words per month.